<a href="https://colab.research.google.com/github/WillemvJ/networks/blob/main/event_driven_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Event-Driven Simulation

Event-driven simulation is a powerful technique used to model complex systems by tracking events that change the state of the system at discrete points in time, rather than continuously. This approach is particularly useful for simulating systems such as queueing systems, network traffic, and industrial processes. The goal of this course is to teach you about modelling logistics networks, and (Event-Driven) Simulation will help us with that.

### What is an Event?

In the context of event-driven simulation, an "event" represents a specific occurrence that changes the state of the system at a particular time. Events are typically scheduled to happen at certain times and are processed sequentially, influencing the system's behavior and the scheduling of future events.

### Types of Events

In this simulation, we will consider two types of events:

1. **Customer Arrival Event**: Represents the arrival of a customer looking to rent a bike. The arrival of a customer potentially triggers the rental of a bike if one is available.
2. **Bike Return Event**: Occurs when a previously rented bike is returned and becomes available for future customers.

### Future Event Set (FES)

The Future Event Set (FES) is a critical component in event-driven simulation. It functions as a priority queue to manage and process events based on the time they occur. The FES ensures that events are handled in chronological order, allowing the simulation to accurately reflect the sequence of actions in the system.

- **Adding an Event**: Events are added to the FES with their scheduled time.
- **Processing an Event**: The system retrieves and processes the next event from the FES (the event with the earliest time). Processing an event may involve changing the state of the system and possibly scheduling new events (e.g., scheduling a bike return when a bike is rented).

This notebook sets up the basic classes and structure to simulate a simple bike rental system using event-driven simulation principles. We will expand on this framework to include handling of bike availability, responding to customer arrivals, and managing bike returns.

After setting up the classes, the notebook includes some code that demonstrates their usage.

In [10]:
import heapq
import numpy as np
import matplotlib.pyplot as plt


class CustomerArrivalEvent:
    def __init__(self, time):
        self.time = time

    def __lt__(self, other):
        return self.time < other.time

    def __str__(self):
        return f"Customer arrival at t = {self.time}"

class BikeReturnEvent:
    def __init__(self, time):
        self.time = time

    def __lt__(self, other):
        return self.time < other.time

    def __str__(self):
        return f"Bike return at t = {self.time}"

# future event set (FES) class
# keeps track of things that will happen at some time
# and allows us to find the first thing to happen.
# from Advanced Simulation course (Marco Boon)
class FutureEventSet:
    def __init__(self):
        self.events = []

    def add(self, event):
        heapq.heappush(self.events, event)

    def next(self):
        return heapq.heappop(self.events)

    def isEmpty(self):
        return len(self.events) == 0

    def __str__(self):
        sortedEvents = sorted(self.events)
        return '\n'.join(str(e) for e in sortedEvents)

## Managing the Future Event Set (FES)

This section demonstrates the the Future Event Set (FES) to keep track of events in a bike rental simulation. We initialize the FES, add events to it, and then process these events sequentially.

### Initialization and Adding Events

First, we initialize the Future Event Set and add several `CustomerArrivalEvent` instances to it. These events represent the times at which customers arrive to rent a bike:

### Adding a Bike Return Event
For illustration purposes, we also add a bike return event to the FES. We then return

In [ ]:
fes = FutureEventSet()
fes.add(CustomerArrivalEvent(5))
fes.add(CustomerArrivalEvent(3))
print("FES after adding two customer arrivals:")
print(fes)

fes.add(BikeReturnEvent(4))
print("\nFES after adding bike return event:")
print(fes)

while not fes.isEmpty():
    print("\nProcessing event:", fes.next())
    print("FES now:")
    print(fes)

### Some actual simulation with the event set

Now, let's do something a bit more usefull with all of this. Suppose we know that at a given day, people arrive at a bike sharing system 1, 1.5, 2.2, and 2.6 hours after opening. The person takes the bike, and returns it after 1 hours later. For ease, let us assume that there are initially 5 bikes. We want to keep track of what happens during the day.

In [ ]:
fes = FutureEventSet()
fes.add(CustomerArrivalEvent(1))
fes.add(CustomerArrivalEvent(1.5))
fes.add(CustomerArrivalEvent(1.6))
fes.add(CustomerArrivalEvent(2.2))
fes.add(CustomerArrivalEvent(2.6))
print(fes)
rental_duration = 1
bikes_available = 2
# Processing events - always first the event with the earliest time
while not fes.isEmpty():
    event = fes.next()
    print()
    print(event)
    if isinstance(event, CustomerArrivalEvent):
        bikes_available -= 1
        return_time = event.time + rental_duration
        # Schedule the bike return, i.e. add it to the set of things that must
        # ``happen''. Note that this may happen before some of the arrival
        # events that are allready scheduled.
        fes.add(BikeReturnEvent(return_time))
    elif isinstance(event, BikeReturnEvent):
        bikes_available += 1
    print("-------FES-------")
    print(fes)
    print("-----------------")
    # this woudl print the amount of bikes available over time.
    # print(f"bikes left {bikes_available}")